In [1]:
import os
from pathlib import Path
from typing import Iterable, Optional

import hydra
import numpy as np
import torch
import wandb
from accelerate import Accelerator
from matplotlib.cm import get_cmap
from omegaconf import DictConfig, OmegaConf
from PIL import Image
from skimage.color import label2rgb
from tqdm import tqdm, trange

/home/guests/oleksandra_tmenova/test/project/.venv/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [21]:
# evaluation utilities
import eval_utils
# for reading a dataset with groundth truth and labels
from dataset import EvalDataset


root_dir = 'demo_dataset'
custom_dataset = EvalDataset(root_dir)

In [22]:
# Add background class
n_classes = 6
n_clusters = 6

# Iterate
tp = [0] * n_classes
fp = [0] * n_classes
fn = [0] * n_classes

# Load all pixel embeddings
all_preds = np.zeros((len(custom_dataset) * 500 * 500), dtype=np.float32)
all_gt = np.zeros((len(custom_dataset) * 500 * 500), dtype=np.float32)
offset_ = 0

In [23]:
for i in trange(len(custom_dataset), desc='Concatenating all predictions'):
    image, target, mask = custom_dataset[i]
    # Check where ground-truth is valid and append valid pixels to the array
    valid = (target != 255)
    n_valid = np.sum(valid)
    all_gt[offset_:offset_+n_valid] = target[valid]
    # Append the predicted targets in the array
    all_preds[offset_:offset_+n_valid, ] = mask[valid]
    all_gt[offset_:offset_+n_valid, ] = target[valid]
    # Update offset_
    offset_ += n_valid

Concatenating all predictions: 100%|██████████| 1/1 [00:00<00:00, 108.21it/s]


In [24]:
# Truncate to the actual number of pixels
all_preds = all_preds[:offset_, ]
all_gt = all_gt[:offset_, ]

# Do hungarian matching
num_elems = offset_
if n_clusters == n_classes:
    print('Using hungarian algorithm for matching')
    match = eval_utils.hungarian_match(all_preds, all_gt, preds_k=n_clusters, targets_k=n_classes, metric='iou')
else:
    print('Using majority voting for matching')
    match = eval_utils.majority_vote(all_preds, all_gt, preds_k=n_clusters, targets_k=n_classes)
print(f'Optimal matching: {match}')

Using hungarian algorithm for matching
Using iou as metric
Optimal matching: [(0, 5), (1, 1), (2, 0), (3, 2), (4, 3), (5, 4)]


In [25]:
# Remap predictions
reordered_preds = np.zeros(num_elems, dtype=all_preds.dtype)
for pred_i, target_i in match:
    reordered_preds[all_preds == int(pred_i)] = int(target_i)

# TP, FP, and FN evaluation
for i_part in range(0, n_classes):
    tmp_all_gt = (all_gt == i_part)
    tmp_pred = (reordered_preds == i_part)
    tp[i_part] += np.sum(tmp_all_gt & tmp_pred)
    fp[i_part] += np.sum(~tmp_all_gt & tmp_pred)
    fn[i_part] += np.sum(tmp_all_gt & ~tmp_pred)

# Calculate Jaccard index
jac = [0] * n_classes
for i_part in range(0, n_classes):
    jac[i_part] = float(tp[i_part]) / max(float(tp[i_part] + fp[i_part] + fn[i_part]), 1e-8)

# Print results
eval_result = dict()
eval_result['jaccards_all_categs'] = jac
eval_result['mIoU'] = np.mean(jac)
print('Evaluation of semantic segmentation ')

Evaluation of semantic segmentation 


In [26]:
eval_result

{'jaccards_all_categs': [0.0,
  0.7693266832917706,
  0.0,
  0.0,
  0.0,
  0.46726946546892595],
 'mIoU': 0.20609935812678273}

In [27]:
match

[(0, 5), (1, 1), (2, 0), (3, 2), (4, 3), (5, 4)]